## Build logistic regression model - Lesson 4

![Logistic vs. linear regression infographic](../../../../../../translated_images/linear-vs-logistic.ba180bf95e7ee66721ba10ebf2dac2666acbd64a88b003c83928712433a13c7d.pcm.png)

#### **[Pre-lecture quiz](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/15/)**

####  Introduction

Dis na di last lesson for Regression, one of di basic *classic* ML techniques. We go look Logistic Regression. You fit use dis technique to find patterns wey go help predict two categories. Dis candy na chocolate or e no be? Dis sickness dey spread or e no dey spread? Dis customer go choose dis product or e no go choose am?

For dis lesson, you go learn:

-   Techniques for logistic regression

✅ Make your understanding strong for how dis regression dey work for dis [Learn module](https://learn.microsoft.com/training/modules/introduction-classification-models/?WT.mc_id=academic-77952-leestott)

## Prerequisite

Since we don work with di pumpkin data, we don sabi am well well to know say e get one binary category wey we fit work with: `Color`.

Make we build logistic regression model to predict, based on some variables, *wetin be di color wey pumpkin go likely be* (orange 🎃 or white 👻).

> Why we dey talk about binary classification for lesson wey dey group under regression? Na just because e dey easy to talk am like dat, as logistic regression na [classification method](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression), even though e dey based on linear regression. You go learn other ways to classify data for di next lesson group.

For dis lesson, we go need di following packages:

-   `tidyverse`: Di [tidyverse](https://www.tidyverse.org/) na [collection of R packages](https://www.tidyverse.org/packages) wey dey make data science fast, easy and fun!

-   `tidymodels`: Di [tidymodels](https://www.tidymodels.org/) framework na [collection of packages](https://www.tidymodels.org/packages/) for modeling and machine learning.

-   `janitor`: Di [janitor package](https://github.com/sfirke/janitor) dey provide simple tools to check and clean dirty data.

-   `ggbeeswarm`: Di [ggbeeswarm package](https://github.com/eclarke/ggbeeswarm) dey provide methods to create beeswarm-style plots using ggplot2.

You fit install dem like dis:

`install.packages(c("tidyverse", "tidymodels", "janitor", "ggbeeswarm"))`

Or, di script wey dey below go check whether you get di packages wey you need for dis module and go install dem for you if dem no dey.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, ggbeeswarm)


## **Define di question**

For wetin we wan do, we go tok am as binary: 'White' or 'Not White'. Di dataset get one 'striped' category but e no too plenty, so we no go use am. E go comot sef once we remove di null values from di dataset.

> 🎃 Fun fact, sometimes we dey call white pumpkins 'ghost' pumpkins. Dem no too easy to carve, so dem no popular like di orange ones but dem dey look cool! So we fit also change di question to: 'Ghost' or 'Not Ghost'. 👻

## **About logistic regression**

Logistic regression no be di same as linear regression wey you don learn before. E get some important difference.

#### **Binary classification**

Logistic regression no dey do di same tin as linear regression. Logistic regression dey predict `binary category` ("orange or not orange") but linear regression fit predict `continual values`, like if you know where pumpkin come from and di time dem harvest am, *how much di price go increase*.

![Infographic by Dasani Madipalli](../../../../../../translated_images/pumpkin-classifier.562771f104ad5436b87d1c67bca02a42a17841133556559325c0a0e348e5b774.pcm.png)

### Other classifications

Logistic regression get other types, like multinomial and ordinal:

- **Multinomial**, wey mean say e get more than one category - "Orange, White, and Striped".

- **Ordinal**, wey mean say e get ordered categories, useful if we wan arrange di outcomes logically, like di pumpkins wey we fit arrange by size (mini, sm, med, lg, xl, xxl).

![Multinomial vs ordinal regression](../../../../../../translated_images/multinomial-vs-ordinal.36701b4850e37d86c9dd49f7bef93a2f94dbdb8fe03443eb68f0542f97f28f29.pcm.png)

#### **Variables NO need to correlate**

You remember how linear regression dey work better when variables dey correlate? Logistic regression no need dat one - di variables no need align. Dis one dey work well for dis data wey no get strong correlations.

#### **You need plenty clean data**

Logistic regression go give better result if you use plenty data; our small dataset no too good for dis task, so make you remember dat one.

✅ Think about di kind data wey go work well for logistic regression

## Exercise - tidy di data

First, clean di data small, remove null values and choose only some columns:

1. Add di following code:


In [ ]:
# Load the core tidyverse packages
library(tidyverse)

# Import the data and clean column names
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv") %>% 
  clean_names()

# Select desired columns
pumpkins_select <- pumpkins %>% 
  select(c(city_name, package, variety, origin, item_size, color)) 

# Drop rows containing missing values and encode color as factor (category)
pumpkins_select <- pumpkins_select %>% 
  drop_na() %>% 
  mutate(color = factor(color))

# View the first few rows
pumpkins_select %>% 
  slice_head(n = 5)


You fit always look ya new dataframe small, if you use [*glimpse()*](https://pillar.r-lib.org/reference/glimpse.html) function like dis:


In [ ]:
pumpkins_select %>% 
  glimpse()


Make we confirm say na true true binary classification problem we go dey do:


In [ ]:
# Subset distinct observations in outcome column
pumpkins_select %>% 
  distinct(color)


### Visualization - categorical plot
By now you don load di pumpkin data again and clean am so e go fit keep dataset wey get some variables, including Color. Make we use ggplot library take visualize di dataframe for di notebook.

Di ggplot library get some beta ways to show your data. For example, you fit compare how di data dey spread for each Variety and Color inside one categorical plot.

1. Create dis kain plot by using di geombar function, use our pumpkin data, and specify color mapping for each pumpkin category (orange or white):


In [ ]:
# Specify colors for each value of the hue variable
palette <- c(ORANGE = "orange", WHITE = "wheat")

# Create the bar plot
ggplot(pumpkins_select, aes(y = variety, fill = color)) +
  geom_bar(position = "dodge") +
  scale_fill_manual(values = palette) +
  labs(y = "Variety", fill = "Color") +
  theme_minimal()

As you dey look di data, you fit see how di Color data take connect wit Variety.

✅ Wit dis kind categorical plot, wetin be some kind interesting tori wey you fit imagine?


### Data pre-processing: feature encoding

Our pumpkin dataset get string values for all di columns. To work wit categorical data dey easy for humans, but e no dey easy for machines. Machine learning algorithms dey work well wit numbers. Na why encoding na very important step for di data pre-processing phase, because e go help us turn categorical data to numerical data, without losing any information. Better encoding fit help us build better model.

For feature encoding, we get two main types of encoders:

1. **Ordinal encoder**: E dey work well for ordinal variables, wey be categorical variables wey their data get logical order, like di `item_size` column for our dataset. E go create mapping wey go make each category get number, wey be di order of di category for di column.

2. **Categorical encoder**: E dey work well for nominal variables, wey be categorical variables wey their data no get logical order, like all di features wey no be `item_size` for our dataset. E dey use one-hot encoding, wey mean say each category go get binary column: di encoded variable go equal 1 if di pumpkin belong to dat Variety, and 0 if e no belong.

Tidymodels get one correct package: [recipes](https://recipes.tidymodels.org/) - na package for preprocessing data. We go define one `recipe` wey go specify say all predictor columns go dey encoded into set of integers, `prep` am to calculate di quantities and statistics wey di operations need, and finally `bake` am to apply di calculations to new data.

> Normally, recipes dey usually dey used as preprocessor for modelling, where e dey define wetin steps dem go apply to one dataset to make am ready for modelling. For dat case, **e dey highly recommend** say make you use `workflow()` instead of to dey manually estimate recipe using prep and bake. We go see all dis one soon.
>
> But for now, we dey use recipes + prep + bake to specify wetin steps dem go apply to one dataset to make am ready for data analysis, and then extract di preprocessed data wit di steps wey dem don apply.


In [ ]:
# Preprocess and extract data to allow some data analysis
baked_pumpkins <- recipe(color ~ ., data = pumpkins_select) %>%
  # Define ordering for item_size column
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  # Convert factors to numbers using the order defined above (Ordinal encoding)
  step_integer(item_size, zero_based = F) %>%
  # Encode all other predictors using one hot encoding
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE) %>%
  prep(data = pumpkin_select) %>%
  bake(new_data = NULL)

# Display the first few rows of preprocessed data
baked_pumpkins %>% 
  slice_head(n = 5)


✅ Wetin be di advantage of using ordinal encoder for di Item Size column?

### Check how di variables take relate

Now we don process our data finish, we fit check how di features and di label take relate so we go fit sabi how di model go take predict di label based on di features. Di best way to do dis kain analysis na to plot di data. 
We go use di ggplot geom_boxplot_ function again, to show how Item Size, Variety and Color take relate for one categorical plot. To make di plot better, we go use di encoded Item Size column and di unencoded Variety column.


In [ ]:
# Define the color palette
palette <- c(ORANGE = "orange", WHITE = "wheat")

# We need the encoded Item Size column to use it as the x-axis values in the plot
pumpkins_select_plot<-pumpkins_select
pumpkins_select_plot$item_size <- baked_pumpkins$item_size

# Create the grouped box plot
ggplot(pumpkins_select_plot, aes(x = `item_size`, y = color, fill = color)) +
  geom_boxplot() +
  facet_grid(variety ~ ., scales = "free_x") +
  scale_fill_manual(values = palette) +
  labs(x = "Item Size", y = "") +
  theme_minimal() +
  theme(strip.text = element_text(size = 12)) +
  theme(axis.text.x = element_text(size = 10)) +
  theme(axis.title.x = element_text(size = 12)) +
  theme(axis.title.y = element_blank()) +
  theme(legend.position = "bottom") +
  guides(fill = guide_legend(title = "Color")) +
  theme(panel.spacing = unit(0.5, "lines"))+
  theme(strip.text.y = element_text(size = 4, hjust = 0)) 


#### Use swarm plot

Because say Color na binary category (White or Not), e need 'one [special way](https://github.com/rstudio/cheatsheets/blob/main/data-visualization.pdf) to take show am for visualization'.

Make we try `swarm plot` to take show how color dey spread with respect to item_size.

We go use [ggbeeswarm package](https://github.com/eclarke/ggbeeswarm) wey dey give methods to create beeswarm-style plots with ggplot2. Beeswarm plots na one way to plot points wey for normal suppose overlap, but e go arrange dem make dem dey side by side.


In [ ]:
# Create beeswarm plots of color and item_size
baked_pumpkins %>% 
  mutate(color = factor(color)) %>% 
  ggplot(mapping = aes(x = color, y = item_size, color = color)) +
  geom_quasirandom() +
  scale_color_brewer(palette = "Dark2", direction = -1) +
  theme(legend.position = "none")


Now wey we don sabi how di binary categories of color and di bigger group of sizes dey relate, make we check logistic regression to fit know di likely color of one pumpkin.

## Build your model

Choose di variables wey you wan use for di classification model and divide di data into training and test sets. [rsample](https://rsample.tidymodels.org/), one package for Tidymodels, dey provide infrastructure wey go make data splitting and resampling easy:


In [ ]:
# Split data into 80% for training and 20% for testing
set.seed(2056)
pumpkins_split <- pumpkins_select %>% 
  initial_split(prop = 0.8)

# Extract the data in each split
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)

# Print out the first 5 rows of the training set
pumpkins_train %>% 
  slice_head(n = 5)


🙌 We don ready to train model by fit di training features to di training label (color).

We go start by create one recipe wey go show di preprocessing steps wey we go do for our data to make am ready for modelling, like: encode categorical variables into set of integers. Just like `baked_pumpkins`, we go create `pumpkins_recipe` but we no go `prep` and `bake` am because e go dey inside workflow, wey you go see for di next few steps.

Plenty ways dey to specify logistic regression model for Tidymodels. Check `?logistic_reg()` For now, we go specify logistic regression model through di default `stats::glm()` engine.


In [ ]:
# Create a recipe that specifies preprocessing steps for modelling
pumpkins_recipe <- recipe(color ~ ., data = pumpkins_train) %>% 
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  step_integer(item_size, zero_based = F) %>%  
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE)

# Create a logistic model specification
log_reg <- logistic_reg() %>% 
  set_engine("glm") %>% 
  set_mode("classification")


Now wey we don get recipe and model specification, we need find way to join dem together inside one object wey go first preprocess di data (prep+bake for back), fit di model on top di preprocessed data, and still allow for post-processing if e dey necessary.

For Tidymodels, dis easy-to-use object na [`workflow`](https://workflows.tidymodels.org/) and e dey carry all your modeling components well well.


In [ ]:
# Bundle modelling components in a workflow
log_reg_wf <- workflow() %>% 
  add_recipe(pumpkins_recipe) %>% 
  add_model(log_reg)

# Print out the workflow
log_reg_wf


Afta dem don *specify* workflow, dem fit use [`fit()`](https://tidymodels.github.io/parsnip/reference/fit.html) function take `train` model. Di workflow go estimate recipe and go preprocess di data before e train, so we no go need do am manually wit prep and bake.


In [ ]:
# Train the model
wf_fit <- log_reg_wf %>% 
  fit(data = pumpkins_train)

# Print the trained workflow
wf_fit


The model print out dey show the coefficients wey e learn during training.

Now we don train the model with the training data, we fit use am predict for the test data using [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Make we start by using the model to predict labels for our test set and the probabilities for each label. If the probability pass 0.5, the predict class na `WHITE`, but if e no reach, na `ORANGE`.


In [ ]:
# Make predictions for color and corresponding probabilities
results <- pumpkins_test %>% select(color) %>% 
  bind_cols(wf_fit %>% 
              predict(new_data = pumpkins_test)) %>%
  bind_cols(wf_fit %>%
              predict(new_data = pumpkins_test, type = "prob"))

# Compare predictions
results %>% 
  slice_head(n = 10)


Very nice! Dis one dey give more insight on how logistic regression dey work.

### Beta understanding wit confusion matrix

To dey compare each prediction wit di "ground truth" actual value no too make sense if you wan sabi how well di model dey predict. Luckily, Tidymodels get some beta tools for hand: [`yardstick`](https://yardstick.tidymodels.org/) - na package wey dem dey use measure how well models dey perform wit performance metrics.

One performance metric wey dem dey use for classification problems na [`confusion matrix`](https://wikipedia.org/wiki/Confusion_matrix). Confusion matrix dey show how well classification model dey perform. E dey show how many examples for each class di model classify correct. For our case, e go show you how many orange pumpkins dem classify as orange and how many white pumpkins dem classify as white; di confusion matrix go also show you how many dem classify for di **wrong** categories.

Di [**`conf_mat()`**](https://tidymodels.github.io/yardstick/reference/conf_mat.html) function from yardstick dey calculate dis cross-tabulation of observed and predicted classes.


In [ ]:
# Confusion matrix for prediction results
conf_mat(data = results, truth = color, estimate = .pred_class)


Make we interpret di confusion matrix. Our model dey try classify pumpkins between two binary categories, category `white` and category `not-white`.

-   If di model talk say pumpkin na white and e really belong to category 'white', we go call am `true positive`, e dey show for di top left number.

-   If di model talk say pumpkin no be white and e really belong to category 'white', we go call am `false negative`, e dey show for di bottom left number.

-   If di model talk say pumpkin na white and e really belong to category 'not-white', we go call am `false positive`, e dey show for di top right number.

-   If di model talk say pumpkin no be white and e really belong to category 'not-white', we go call am `true negative`, e dey show for di bottom right number.

| Truth |
|:-----:|


|               |        |       |
|---------------|--------|-------|
| **Predicted** | WHITE | ORANGE |
| WHITE        | TP     | FP    |
| ORANGE         | FN     | TN    |

As you fit guess, e better make we get plenty true positives and true negatives, and make di false positives and false negatives small, because e mean say di model dey perform well.

Di confusion matrix dey useful because e dey help us calculate other metrics wey fit help us check how well di classification model dey perform. Make we look some of dem:

🎓 Precision: `TP/(TP + FP)` e mean di proportion of di things wey di model predict as positive wey really be positive. Dem dey also call am [positive predictive value](https://en.wikipedia.org/wiki/Positive_predictive_value "Positive predictive value").

🎓 Recall: `TP/(TP + FN)` e mean di proportion of di positive results out of di number of samples wey really be positive. Dem dey also call am `sensitivity`.

🎓 Specificity: `TN/(TN + FP)` e mean di proportion of di negative results out of di number of samples wey really be negative.

🎓 Accuracy: `TP + TN/(TP + TN + FP + FN)` Na di percentage of di labels wey di model predict correct for di samples.

🎓 F Measure: Na di weighted average of di precision and recall, di best na 1 and di worst na 0.

Make we calculate all dis metrics!


In [ ]:
# Combine metric functions and calculate them all at once
eval_metrics <- metric_set(ppv, recall, spec, f_meas, accuracy)
eval_metrics(data = results, truth = color, estimate = .pred_class)


## Show di ROC curve for dis model

Make we do one more visualization to see wetin dem dey call [`ROC curve`](https://en.wikipedia.org/wiki/Receiver_operating_characteristic):


In [ ]:
# Make a roc_curve
results %>% 
  roc_curve(color, .pred_ORANGE) %>% 
  autoplot()


ROC curves dey often dey use to take see how classifier dey perform in terms of true positives versus false positives. ROC curves dey usually show `True Positive Rate`/Sensitivity for Y axis, and `False Positive Rate`/1-Specificity for X axis. So, how steep di curve be and di space wey dey between di middle line and di curve dey important: you go wan make di curve quick quick go up and pass di line. For our case, false positives dey first, but di line later go up and pass di line well.

Last last, make we use `yardstick::roc_auc()` to calculate di real Area Under di Curve. One way wey you fit take understand AUC na as di chance say di model go rank one random positive example higher pass one random negative example.


In [ ]:
# Calculate area under curve
results %>% 
  roc_auc(color, .pred_ORANGE)


Di result na around `0.975`. Since AUC dey range from 0 to 1, you go wan make di score big, because model wey dey 100% correct for im predictions go get AUC of 1; for dis case, di model *dey very good*.

For future lessons on classifications, you go learn how you fit take improve your model scores (like how to handle imbalanced data for dis kind case).

## 🚀Challenge

Plenty tins dey to learn about logistic regression! But di best way to sabi am na to try am. Find one dataset wey fit work for dis kind analysis and build one model with am. Wetin you learn? Tip: try [Kaggle](https://www.kaggle.com/search?q=logistic+regression+datasets) to find interesting datasets.

## Review & Self Study

Read di first few pages of [dis paper from Stanford](https://web.stanford.edu/~jurafsky/slp3/5.pdf) wey talk about some practical ways to use logistic regression. Think about di tasks wey go fit one type of regression pass di other one wey we don study so far. Which one go work best?


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Disclaimer**:  
Dis dokyument don use AI translation service [Co-op Translator](https://github.com/Azure/co-op-translator) do di translation. Even as we dey try make am accurate, abeg make you sabi say machine translation fit get mistake or no dey correct well. Di original dokyument wey dey for im native language na di main source wey you go trust. For important information, e better make professional human translation dey use. We no go fit take blame for any misunderstanding or wrong interpretation wey fit happen because you use dis translation.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
